<a href="https://colab.research.google.com/github/adeepH/SPDRA-2021-SharedTask/blob/main/Notebooks/Inference_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/adeepH/SPDRA-2021-SharedTask.git 

In [ ]:
cd /content/SPDRA-2021-SharedTask/

In [ ]:
!pip install -r requirements.txt

In [ ]:
cd scripts/

In [ ]:
import pandas as pd
import numpy as np
import torch
from dataset import *
from model import *
from utils import *

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('allenai/scibert_scivocab_uncased') 
device = 'cuda'
MAX_LEN = 128
BATCH_SIZE = 32

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
model = RFClassifier(7,'allenai/scibert_scivocab_uncased')
model.load_state_dict(torch.load('/content/drive/MyDrive/spdra2021/saved_models/scibert.bin'))
model.to(device)
for param in model.parameters():
    param.requires_grad = False
model.eval()

In [ ]:
val = pd.read_csv('/content/drive/MyDrive/spdra2021/Datasets/validation.csv',delimiter=',',
                 header=None,names=['text','label'])

val.label = val.label.apply({'CL':0,'CR':1,'DC':2,
'DS':3,'LO':4, 'NI':5, 'SE':6}.get)

In [ ]:
!python dataset.py

In [ ]:
val_data_loader = create_data_loader(val,tokenizer,MAX_LEN,BATCH_SIZE)

In [ ]:
y_review_texts, y_pred, y_pred_probs, y_test = get_predictions(
  model,
  val_data_loader
)

In [71]:
 
class_name = ['CL' ,'CR' ,'DC' ,'DS' ,'LO' , 'NI' , 'SE' ]
from sklearn.metrics import classification_report,confusion_matrix
print(classification_report(y_test, y_pred, target_names=class_name,zero_division=0))

              precision    recall  f1-score   support

          CL       0.98      0.98      0.98      1866
          CR       0.91      0.90      0.90      1835
          DC       0.80      0.79      0.80      1355
          DS       0.91      0.92      0.92      1774
          LO       0.90      0.92      0.91      1217
          NI       0.91      0.89      0.90      1826
          SE       0.89      0.90      0.89      1327

    accuracy                           0.90     11200
   macro avg       0.90      0.90      0.90     11200
weighted avg       0.90      0.90      0.90     11200



In [76]:
def get_preds(model,data_loader):

    sentence    = []
    predictions = []
    probs_preds = []
    real_values = []

    with torch.no_grad():
        for d in data_loader:
            
            texts = d["text"]
            input_ids = d["input_ids"].to(device)
            attention_mask = d["attention_mask"].to(device)
            labels = d["label"].to(device)
            
            outputs = model(
                input_ids = input_ids,
                attention_mask = attention_mask
            )

            _,preds = torch.max(outputs,dim=1)
            probs = nn.functional.softmax(outputs,dim=1)
            
            sentence.extend(texts)
            predictions.extend(preds)
            probs_preds.extend(probs)
            real_values.extend(labels)

    predictions = torch.stack(predictions).cpu().detach().numpy().tolist()
    probs_preds = torch.stack(probs_preds).cpu().detach().numpy().tolist()
    real_values = torch.stack(real_values).cpu().detach().numpy().tolist()
    
    return sentence,predictions,probs_preds,real_values
            


In [77]:
abstract,y_pred,y_prob,y_test = get_preds(model,val_data_loader)

In [78]:
y_prob[2],y_test[2],y_pred[2]

([0.9981364011764526,
  0.00018715395708568394,
  4.0024242480285466e-05,
  0.00010960720101138577,
  0.00038874585879966617,
  0.00011496632941998541,
  0.0010230711195617914],
 0,
 0)

In [1]:
pred_probs = pd.DataFrame(data = (abstract,y_prob),columns=class_name)
pred_probs['abstract'] = abstract
pred_probs.head()

NameError: ignored